# ARC Grid Forensics Toolkit - Production Solver

## Overview
This notebook implements a production-grade ARC (Abstraction and Reasoning Corpus) solver using a modular, forensics-inspired architecture.

## Architecture Components
1. **arc_utils.py** - Core data structures, I/O, and visualization
2. **grid_forensics.py** - Object detection and grid analysis (BFS flood-fill)
3. **hypothesis_engine.py** - Rule framework and pattern learning
4. **arc_solver_orchestrator.py** - Main execution controller

## Execution Flow
```
Load Data → Analyze Grids → Learn Rules → Apply Transformations → Generate Submission
```

---
## Cell 1: Dependencies and Environment Setup

In [ ]:
# Install optional dependencies (if needed)
# !pip install numpy tqdm -q

import logging
import json
import os
import time
import numpy as np
from pathlib import Path
from typing import List, Dict, Any, Tuple, Set, Optional
from dataclasses import dataclass, field
from collections import deque
from abc import ABC, abstractmethod

# Attempt to import tqdm for progress visualization
try:
    from tqdm import tqdm
except ImportError:
    def tqdm(iterable, *args, **kwargs):
        return iterable
    print("⚠️ tqdm not found. Using simple iteration.")

# Configure global logging
logging.basicConfig(
    level=logging.INFO,  # Change to DEBUG for detailed tracing
    format='[%(asctime)s] {%(levelname)s} [%(name)s]: %(message)s',
    datefmt='%H:%M:%S'
)

logger = logging.getLogger(__name__)
logger.info("✅ Environment initialized successfully")

---
## Cell 2: Module 1 - arc_utils.py
**Purpose**: Core data structures, JSON loading, and grid visualization

In [ ]:
"""
ARC Utility and Data Structure Module (arc_utils.py) - Production Version

This module handles core ARC competition functionality:
1. Data structures for tasks and examples (using dataclasses).
2. Robust JSON file loading and validation.
3. Grid visualization utility (for quick debugging).

This is the I/O layer of the solver.
"""

logger = logging.getLogger('arc_utils')

# --- Core Data Structures (The Schema) ---

@dataclass(frozen=True)
class ArcExample:
    """Represents a single input/output example within an ARC task."""
    input: List[List[int]]
    output: List[List[int]]

@dataclass(frozen=True)
class ArcTask:
    """
    Represents a full ARC task, comprising training examples and test examples.
    This structure ensures all tasks conform to the expected competition format.
    """
    train: List[ArcExample]
    test: List[ArcExample]
    task_id: str = field(default="")  # Unique ID for this task (e.g., "007d3886")
    
    def __post_init__(self):
        """Perform basic validation on load."""
        if not self.train:
            logger.warning(f"Task {self.task_id}: No training examples found.")
        if not self.test:
            logger.warning(f"Task {self.task_id}: No test examples found.")

# --- File Loading and Parsing ---

def get_arc_data_path(base_path: str = ".") -> Path:
    """
    Attempts to locate the root ARC data directory, adapting to common
    competition environments (like Kaggle or local development).
    
    Args:
        base_path: The directory to start searching from.
        
    Returns:
        The Path object pointing to the data directory.
        
    Raises:
        FileNotFoundError: If the data directory cannot be located.
    """
    # Kaggle-specific paths
    kaggle_paths = [
        Path("/kaggle/input/arc-prize-2024"),
        Path("/kaggle/input/arc-agi-2024"),
        Path("/kaggle/input/abstraction-and-reasoning-challenge"),
    ]
    
    # Check Kaggle paths first
    for path in kaggle_paths:
        if path.exists():
            logger.info(f"📁 ARC data path located at: {path.resolve()}")
            return path
    
    # Local development paths
    arc_dir = Path(base_path) / "arc-prize-2024"
    if not arc_dir.exists():
        arc_dir = Path(base_path) / "arc-agi-2024"
        if not arc_dir.exists():
            arc_dir = Path(base_path)
            if not (arc_dir / "arc-agi_training_challenges.json").exists():
                logger.error(f"Could not find ARC data directory starting from {base_path}.")
                raise FileNotFoundError("ARC data directory not found. Check environment setup.")
    
    logger.info(f"📁 ARC data path located at: {arc_dir.resolve()}")
    return arc_dir

def load_arc_tasks(filepath: Path) -> Dict[str, ArcTask]:
    """
    Loads ARC tasks from a JSON file and structures them using ArcTask objects.
    
    Args:
        filepath: Path object pointing to the JSON file (e.g., train_challenges.json).
        
    Returns:
        A dictionary mapping task IDs (str) to ArcTask instances.
        
    Raises:
        FileNotFoundError: If the specified file does not exist.
        json.JSONDecodeError: If the file is not valid JSON.
    """
    if not filepath.exists():
        logger.error(f"load_arc_tasks: File not found at {filepath.resolve()}")
        raise FileNotFoundError(f"Input file not found: {filepath}")

    logger.info(f"📂 Loading and parsing {filepath.name}...")
    
    try:
        with open(filepath, 'r') as f:
            raw_data = json.load(f)
    except json.JSONDecodeError as e:
        logger.error(f"load_arc_tasks: Failed to decode JSON from {filepath.name}. Error: {e}")
        raise
        
    parsed_tasks: Dict[str, ArcTask] = {}
    
    for task_id, task_data in raw_data.items():
        try:
            train_examples = [
                ArcExample(input=ex['input'], output=ex['output'])
                for ex in task_data['train']
            ]
            test_examples = [
                ArcExample(input=ex['input'], output=ex['output'])
                for ex in task_data['test']
            ]
            
            parsed_tasks[task_id] = ArcTask(
                train=train_examples, 
                test=test_examples, 
                task_id=task_id
            )
            
        except KeyError as e:
            logger.error(f"Task {task_id} is missing a required key: {e}. Skipping.")
        except Exception as e:
            logger.error(f"Failed to parse task {task_id}. Error: {e}. Skipping.")

    logger.info(f"✅ Successfully loaded and parsed {len(parsed_tasks)} tasks.")
    return parsed_tasks

# --- Visualization Utility (for debugging) ---

# Custom ANSI color map for clear console display
# Colors 0-9 correspond to ARC's palette
ANSI_COLORS = {
    0: "\033[47m  \033[0m",  # White/Background (using bright white BG)
    1: "\033[44m  \033[0m",  # Blue
    2: "\033[41m  \033[0m",  # Red
    3: "\033[42m  \033[0m",  # Green
    4: "\033[43m  \033[0m",  # Yellow
    5: "\033[40m  \033[0m",  # Black
    6: "\033[45m  \033[0m",  # Magenta
    7: "\033[46m  \033[0m",  # Cyan
    8: "\033[100m  \033[0m", # Gray (Bright Black BG)
    9: "\033[105m  \033[0m", # Orange/Purple (Bright Magenta BG)
}

def display_grid(grid: List[List[int]], title: str = "Grid View") -> None:
    """
    Prints a grid to the console with color-coded ANSI characters.
    This provides an immediate, visual debugging tool.
    
    Args:
        grid: The 2D list grid to display.
        title: The title to display above the grid.
    """
    if not grid or not grid[0]:
        print(f"\n--- {title} (EMPTY) ---")
        return
        
    print(f"\n--- {title} ({len(grid)}x{len(grid[0])}) ---")
    
    output = []
    for row in grid:
        line = "".join(ANSI_COLORS.get(cell, f"[{cell}]") for cell in row)
        output.append(line)
    
    print("\n".join(output))
    print("-" * (len(title) + 12))

logger.info("✅ arc_utils module loaded")

---
## Cell 3: Module 2 - grid_forensics.py
**Purpose**: Object detection using BFS flood-fill and property extraction

In [ ]:
"""
Grid Forensics Toolkit (grid_forensics.py) - Production Version

This module provides the core "disassembly" tools for ARC grids.
It is designed to be a stable, reliable, and performant library for
turning raw pixel grids (the "binary") into structured, object-oriented
reports (the "assembly").

This module is the "IDA Pro" or "Volatility" for your solver. It provides
the static analysis that all higher-level "vulnerability" (hypothesis)
research will be built upon.

Main Public API:
- class GridAnalysis(grid, background_color=0)
- func analyze_grid(grid, background_color=0)
"""

logger = logging.getLogger('grid_forensics')

# --- Core Object Definition ---

class ArcObject:
    """
    Represents a single, contiguous object (structure) found in a grid.
    This is the primary unit of analysis, analogous to a process,
    file handle, or data structure in memory forensics.
    """
    
    def __init__(self, color: int, coords: Set[Tuple[int, int]]):
        """
        Initializes an object with its color and a set of (row, col) coordinates.
        """
        self.color: int = color
        self.coords: Set[Tuple[int, int]] = coords
        self.size: int = len(coords)
        
        # Properties are lazy-loaded by get_object_properties
        self.bounding_box: Optional[Tuple[int, int, int, int]] = None  # (min_r, min_c, max_r, max_c)
        self.height: int = 0
        self.width: int = 0
        self.shape_hash: str = ""  # String "mask" of the shape (e.g., "1101")
        self.grid_array: Optional[np.ndarray] = None  # Minimal numpy array of the object

    def __repr__(self) -> str:
        """Provides a clean, readable string for debugging."""
        return f"ArcObject(Color={self.color}, Size={self.size}, BB={self.bounding_box})"

    def to_dict(self) -> Dict[str, Any]:
        """Serializes the object's properties for JSON logging or reports."""
        return {
            "color": self.color,
            "size": self.size,
            "bounding_box": self.bounding_box,
            "height": self.height,
            "width": self.width,
            "shape_hash": self.shape_hash,
            "grid_array": self.grid_array.tolist() if self.grid_array is not None else None
        }

# --- Core "Disassembly" Functions ---

def find_objects(grid: np.ndarray, ignore_color: int = 0) -> List[ArcObject]:
    """
    Finds all contiguous objects in a grid using a Breadth-First Search (BFS)
    flood-fill algorithm. This is the "structure-finding" part of the
    disassembler (like finding function prologs in IDA Pro).
    
    Args:
        grid: The 2D numpy array to scan.
        ignore_color: The color to treat as "background" (usually 0).
                      These pixels will not be part of any object.
                      
    Returns:
        A list of ArcObject instances.
    """
    logger.debug(f"find_objects: Starting scan on grid {grid.shape}. Ignoring color {ignore_color}.")
    height, width = grid.shape
    visited = np.zeros_like(grid, dtype=bool)
    objects: List[ArcObject] = []
    
    # Use a deque for an efficient O(1) queue for the BFS
    q: deque[Tuple[int, int]] = deque()
    
    for r in range(height):
        for c in range(width):
            # Skip if visited or if it's the background/ignored color
            if visited[r, c] or grid[r, c] == ignore_color:
                visited[r, c] = True
                continue

            # New object found. Start flood fill.
            color = grid[r, c]
            coords: Set[Tuple[int, int]] = set()
            
            q.append((r, c))
            visited[r, c] = True
            
            while q:
                curr_r, curr_c = q.popleft()
                coords.add((curr_r, curr_c))
                
                # Check 4-way neighbors
                for dr, dc in [(0, 1), (0, -1), (1, 0), (-1, 0)]:
                    nr, nc = curr_r + dr, curr_c + dc
                    
                    # Check boundaries
                    if 0 <= nr < height and 0 <= nc < width:
                        # Check color and if already visited
                        if not visited[nr, nc] and grid[nr, nc] == color:
                            visited[nr, nc] = True
                            q.append((nr, nc))
                            
            if coords:
                logger.debug(f"  > Found object: color={color}, size={len(coords)} at ({r},{c})")
                objects.append(ArcObject(color, coords))
                
    logger.debug(f"find_objects: Scan complete. Found {len(objects)} objects.")
    return objects

def get_object_properties(obj: ArcObject, background_color: int = -1) -> None:
    """
    Analyzes a found ArcObject to extract its core properties.
    This is the "property dumper" (like `procdump` or IDA's structure view).
    
    This function modifies the object IN-PLACE.
    
    Args:
        obj: The ArcObject to analyze.
        background_color: The color to use as the "fill" for the
                          minimal `grid_array` (default -1).
    """
    if not obj.coords:
        logger.warning(f"get_object_properties: Called on an empty object.")
        return

    # 1. Calculate Bounding Box
    min_r = min(r for r, c in obj.coords)
    min_c = min(c for r, c in obj.coords)
    max_r = max(r for r, c in obj.coords)
    max_c = max(c for r, c in obj.coords)
    
    obj.bounding_box = (min_r, min_c, max_r, max_c)
    obj.height = (max_r - min_r) + 1
    obj.width = (max_c - min_c) + 1

    # 2. Generate minimal grid_array (the object's "footprint")
    # This is a *highly* useful property for matching, rotation, etc.
    obj.grid_array = np.full((obj.height, obj.width), background_color, dtype=int)
    shape_mask = np.zeros((obj.height, obj.width), dtype=int)
    
    for r, c in obj.coords:
        norm_r, norm_c = r - min_r, c - min_c
        obj.grid_array[norm_r, norm_c] = obj.color
        shape_mask[norm_r, norm_c] = 1
        
    # 3. Generate a "Shape Hash"
    # This is a string "mask" of the shape, normalized to its top-left corner.
    # e.g., a 2x2 square is "1111". A 2x2 'L' shape is "1011".
    obj.shape_hash = "".join(map(str, shape_mask.flatten()))

# --- Main Public API ---

def analyze_grid(grid: List[List[int]], background_color: int = 0) -> Dict[str, Any]:
    """
    Main entry point for "Grid Forensics".
    Takes a raw grid and returns a full "autopsy report"
    that the solver can use to make intelligent decisions.
    
    Args:
        grid: The raw 2D list grid from the ARC task.
        background_color: The color to treat as background (default 0).
        
    Returns:
        A dictionary report containing all analysis.
    """
    if not grid or not isinstance(grid, list) or not grid[0]:
        logger.warning("analyze_grid: Received an empty or invalid grid.")
        return {
            "height": 0, "width": 0, "total_pixels": 0,
            "unique_colors": [], "object_count": 0, "objects": [],
            "error": "Empty or invalid grid"
        }
        
    try:
        grid_np = np.array(grid, dtype=int)
    except Exception as e:
        logger.error(f"analyze_grid: Could not convert grid to numpy array. Error: {e}")
        return {
            "height": 0, "width": 0, "total_pixels": 0,
            "unique_colors": [], "object_count": 0, "objects": [],
            "error": f"Numpy conversion failed: {e}"
        }
    
    # 1. Find all object structures
    objects = find_objects(grid_np, ignore_color=background_color)
    
    # 2. Get properties for each object
    for obj in objects:
        get_object_properties(obj)  # Modifies obj in-place
        
    # 3. Generate the final report
    report = {
        "height": grid_np.shape[0],
        "width": grid_np.shape[1],
        "total_pixels": int(grid_np.size),
        "unique_colors": [int(c) for c in np.unique(grid_np)],
        "object_count": len(objects),
        "objects": objects  # The list of ArcObject instances
    }
    
    logger.debug(f"analyze_grid: Analysis complete. Found {len(objects)} objects.")
    return report

class GridAnalysis:
    """
    A high-level convenience wrapper class that holds the raw grid,
    the numpy grid, and the full analysis report.
    """
    def __init__(self, grid: List[List[int]], background_color: int = 0):
        self.raw_grid = grid
        self.background_color = background_color
        
        try:
            self.grid_np = np.array(grid, dtype=int)
            self.report = analyze_grid(grid, background_color)
        except Exception as e:
            logger.error(f"GridAnalysis __init__: Failed. Error: {e}")
            self.grid_np = np.array([[]], dtype=int)
            self.report = {"error": str(e), "objects": []}

        self.objects: List[ArcObject] = self.report.get("objects", [])
        self.object_count: int = self.report.get("object_count", 0)
        self.height: int = self.report.get("height", 0)
        self.width: int = self.report.get("width", 0)

    def to_dict(self) -> Dict[str, Any]:
        """Serializes the full report for logging."""
        serializable_report = self.report.copy()
        serializable_report["objects"] = [o.to_dict() for o in self.objects]
        return serializable_report

logger.info("✅ grid_forensics module loaded")

---
## Cell 4: Module 3 - hypothesis_engine.py
**Purpose**: Rule framework for pattern learning and transformation

In [ ]:
"""
Hypothesis Engine Architecture (hypothesis_engine.py) - Production Version

This module defines the abstract framework for creating and applying
transformation rules. It acts as the orchestration layer between the
GridAnalysis (static analysis) and the actual solution generation.

Core Components:
- BaseRule: Abstract class defining the required API for any rule.
- HypothesisEngine: The main class that loads, trains, and applies rules.
"""

logger = logging.getLogger('hypothesis_engine')

# --- Rule Interface (The Plugin API) ---

class BaseRule(ABC):
    """
    Abstract Base Class for all ARC transformation rules.
    Every rule must implement the 'learn' and 'apply' methods.
    """
    
    # REQUIRED: A short, unique identifier for the rule
    RULE_ID: str = "BaseRule"
    
    def __init__(self):
        """Initializes the rule with state storage."""
        self.is_learned: bool = False
        self.metadata: Dict[str, Any] = {}
        logger.debug(f"Rule {self.RULE_ID} initialized.")

    @abstractmethod
    def learn(self, task: ArcTask) -> bool:
        """
        Analyzes the training examples in an ArcTask to determine if this rule
        can explain the transformation (Input -> Output).
        
        Args:
            task: The ArcTask containing training data.
            
        Returns:
            True if the rule successfully learned a pattern, False otherwise.
        """
        pass

    @abstractmethod
    def apply(self, input_grid: List[List[int]]) -> Optional[List[List[int]]]:
        """
        Applies the learned transformation to a new input grid.
        
        Args:
            input_grid: The raw 2D input grid.
            
        Returns:
            The transformed output grid (List[List[int]]) or None if the
            rule cannot be applied or is not learned.
        """
        pass

# --- Rule Implementations (Example: The simplest rule) ---

class ColorSwapRule(BaseRule):
    """
    A simple rule that checks if the task is a single color swap.
    Example: All 1s become 2s.
    """
    RULE_ID = "ColorSwap_R01"

    def __init__(self):
        super().__init__()
        # Internal state to store the required color mapping
        self.mapping: Dict[int, int] = {}
        self.background_color: int = 0
        
    def learn(self, task: ArcTask) -> bool:
        """
        Learns the color swap mapping based on the first example.
        """
        if not task.train:
            return False
            
        example = task.train[0]
        
        # 1. Analyze input and output colors
        input_analysis = GridAnalysis(example.input, self.background_color)
        output_analysis = GridAnalysis(example.output, self.background_color)
        
        input_colors = set(input_analysis.report['unique_colors'])
        output_colors = set(output_analysis.report['unique_colors'])
        
        # Remove background color from consideration
        if self.background_color in input_colors:
            input_colors.remove(self.background_color)
        if self.background_color in output_colors:
            output_colors.remove(self.background_color)

        # Basic check: The number of unique non-background colors must match
        if len(input_colors) != len(output_colors) or not input_colors:
            logger.debug(f"{self.RULE_ID}: Color count mismatch or no objects.")
            return False

        # 2. Derive the mapping
        # Simplistic mapping: smallest input color maps to smallest output color, etc.
        # This is a placeholder for a much more complex mapping
        sorted_input = sorted(list(input_colors))
        sorted_output = sorted(list(output_colors))
        
        self.mapping = dict(zip(sorted_input, sorted_output))
        
        if not self.mapping:
             return False
        
        # 3. Validation: Test the mapping on all training examples
        for ex in task.train:
            predicted_output = self._perform_swap(ex.input)
            if predicted_output != ex.output:
                logger.debug(f"{self.RULE_ID}: Failed validation on a training example.")
                self.mapping = {}  # Reset state
                return False

        # If all checks pass, we learned the rule
        self.is_learned = True
        self.metadata['mapping'] = self.mapping
        logger.info(f"{self.RULE_ID}: Successfully learned mapping {self.mapping}.")
        return True

    def _perform_swap(self, grid: List[List[int]]) -> List[List[int]]:
        """Helper to apply the internal mapping to a grid."""
        output_grid = []
        for row in grid:
            new_row = [self.mapping.get(cell, cell) for cell in row]
            output_grid.append(new_row)
        return output_grid

    def apply(self, input_grid: List[List[int]]) -> Optional[List[List[int]]]:
        """Applies the learned color swap."""
        if not self.is_learned:
            logger.warning(f"{self.RULE_ID}: Cannot apply, rule not learned.")
            return None
        
        return self._perform_swap(input_grid)

# --- The Orchestration Layer (The main component) ---

class HypothesisEngine:
    """
    The central intelligence unit that loads available rules, attempts
    to train them on a given task, and then uses the best-fit rules
    to generate predictions.
    """
    
    def __init__(self, rule_classes: List[type[BaseRule]]):
        """
        Initializes the engine by registering all available rule classes.
        
        Args:
            rule_classes: A list of classes inheriting from BaseRule.
        """
        self.rule_classes = rule_classes
        self.learned_rules: List[BaseRule] = []
        logger.info(f"HypothesisEngine initialized with {len(rule_classes)} rule types.")

    def train_task(self, task: ArcTask) -> List[BaseRule]:
        """
        Attempts to train all registered rule types on the given task.
        
        Args:
            task: The ArcTask to train on.
            
        Returns:
            A list of successfully learned (trained) BaseRule instances.
        """
        self.learned_rules = []
        logger.info(f"--- Training on Task: {task.task_id} ---")

        for RuleClass in self.rule_classes:
            try:
                rule = RuleClass()
                logger.debug(f"Attempting to learn with Rule: {rule.RULE_ID}...")
                
                if rule.learn(task):
                    self.learned_rules.append(rule)
                    logger.info(f"✅ Success: Rule {rule.RULE_ID} learned the pattern.")
                else:
                    logger.debug(f"❌ Failure: Rule {rule.RULE_ID} did not match.")
                    
            except Exception as e:
                logger.error(f"FATAL: Rule {RuleClass.RULE_ID} crashed during learn phase: {e}", exc_info=True)

        if not self.learned_rules:
            logger.warning(f"No rules could explain Task {task.task_id}. Proceeding without a strong hypothesis.")
        
        return self.learned_rules

    def solve_test_case(self, input_grid: List[List[int]]) -> List[List[int]]:
        """
        Uses the list of successfully learned rules to generate a prediction.
        
        In a production solver, this would try the highest-confidence rule first.
        For now, we just return the first successful prediction.
        
        Args:
            input_grid: The test input grid.
            
        Returns:
            The predicted output grid, or the original input grid as a fallback.
        """
        if not self.learned_rules:
            logger.warning("No hypotheses available to solve. Returning identity transformation.")
            return input_grid  # Identity transformation (the safest guess)

        # Attempt to apply rules in the order they were learned (or by priority/score later)
        for rule in self.learned_rules:
            logger.debug(f"Attempting to apply learned rule: {rule.RULE_ID}")
            prediction = rule.apply(input_grid)
            
            if prediction is not None:
                logger.info(f"✅ Prediction generated by Rule: {rule.RULE_ID}")
                return prediction
        
        logger.warning("All learned rules failed to generate a prediction. Returning identity transformation.")
        return input_grid

logger.info("✅ hypothesis_engine module loaded")

---
## Cell 5: Module 4 - arc_solver_orchestrator.py
**Purpose**: Main solver orchestrator and submission generator

In [ ]:
"""
ARC Solver Orchestrator (arc_solver_orchestrator.py) - Production Version

This is the main entry point and execution controller for the ARC Solver.

It performs the following high-level tasks:
1. Loads all challenge data (train/test).
2. Initializes the Hypothesis Engine with all known transformation rules.
3. Iterates through each task, training the engine and generating predictions.
4. Formats and saves the final submission file.
"""

logger = logging.getLogger('arc_solver')

# --- Solver Orchestrator Class ---

class ArcSolver:
    """
    Manages the entire ARC solving process from data loading to submission generation.
    """
    
    def __init__(self, data_root: str = ".", submission_path: str = "submission.json"):
        """
        Initializes the solver, registers all rules, and sets up paths.
        
        Args:
            data_root: Base directory to search for ARC data.
            submission_path: Path to save the final submission JSON file.
        """
        self.data_root = data_root
        self.submission_path = Path(submission_path)
        self.challenges: Dict[str, ArcTask] = {}
        self.predictions: Dict[str, List[List[List[int]]]] = {}
        
        # Register all available rules here. This list will grow over time.
        self.available_rules: List[type[BaseRule]] = [
            ColorSwapRule, 
            # Future rules (e.g., RotationRule, FloodFillRule) would be added here
        ]
        
        # Initialize the core rule engine
        self.engine = HypothesisEngine(rule_classes=self.available_rules)
        logger.info(f"🚀 ArcSolver initialized. Rules registered: {[r.RULE_ID for r in self.engine.rule_classes]}")

    def load_data(self, dataset_name: str = "arc-agi_test_challenges.json") -> bool:
        """
        Locates the data path and loads the specified challenges file.
        
        Args:
            dataset_name: The name of the JSON file containing the tasks.
            
        Returns:
            True if data was successfully loaded, False otherwise.
        """
        try:
            arc_data_dir = get_arc_data_path(self.data_root)
            data_filepath = arc_data_dir / dataset_name
            self.challenges = load_arc_tasks(data_filepath)
            logger.info(f"✅ Successfully loaded {len(self.challenges)} tasks.")
            return True
        except FileNotFoundError as e:
            logger.error(f"❌ FATAL: Data loading failed. {e}")
            return False
        except Exception as e:
            logger.error(f"❌ FATAL: An unexpected error occurred during data loading: {e}", exc_info=True)
            return False

    def run_solver(self, max_attempts_per_case: int = 1, visualize_first: bool = False):
        """
        Iterates through all loaded tasks, trains the engine, and makes predictions.
        
        Args:
            max_attempts_per_case: Max number of predictions to generate per test example.
                                   ARC submission allows up to 3 predictions.
            visualize_first: If True, displays the first task's input/output.
        """
        if not self.challenges:
            logger.warning("⚠️ No challenges loaded. Please run load_data() first.")
            return

        total_tasks = len(self.challenges)
        logger.info(f"🔄 Starting solver loop for {total_tasks} tasks...")
        
        for task_id, task in tqdm(self.challenges.items(), desc="Solving Tasks"):
            predictions_for_task = []
            
            # Phase 1: Training (Find the pattern)
            learned_rules = self.engine.train_task(task)
            
            # If no rule was learned, skip prediction generation for this task.
            if not learned_rules:
                self.predictions[task_id] = predictions_for_task
                continue

            # Phase 2: Prediction (Apply the pattern to test cases)
            for i, example in enumerate(task.test):
                test_input = example.input
                # We can try to use different rules/strategies for multiple attempts
                
                for attempt in range(max_attempts_per_case):
                    # For simplicity, we currently rely on the engine's internal priority
                    predicted_output = self.engine.solve_test_case(test_input)
                    
                    if predicted_output != test_input:  # Check if a real transformation occurred
                        predictions_for_task.append(predicted_output)
                        logger.debug(f"Task {task_id}, Test {i}: Generated prediction {attempt + 1}")
                        break  # Move to the next test case
                        
            # Store the final list of predictions for this task
            self.predictions[task_id] = predictions_for_task
            
            # Optional: Display the first solved test case for visual confirmation
            if visualize_first and predictions_for_task and len(task.test) > 0:
                display_grid(task.test[0].input, f"Task {task_id} Test Input")
                display_grid(predictions_for_task[0], f"Task {task_id} Predicted Output")
                visualize_first = False  # Only show first task

    def generate_submission(self) -> str:
        """
        Formats the predictions into the required submission JSON structure.
        The required format is: {"task_id": [output_grid_1, output_grid_2, ...]}
        
        Returns:
            The path to the generated submission file.
        """
        submission_data = {}
        for task_id, output_list in self.predictions.items():
            submission_data[task_id] = output_list

        try:
            with open(self.submission_path, 'w') as f:
                json.dump(submission_data, f, indent=2)
            logger.info(f"💾 Submission file successfully written to {self.submission_path.resolve()}")
            return str(self.submission_path.resolve())
        except Exception as e:
            logger.error(f"❌ Failed to write submission file: {e}")
            return f"Error: {e}"

logger.info("✅ arc_solver_orchestrator module loaded")

---
## Cell 6: Main Execution - Run the Solver
**This is the main cell to execute the entire pipeline**

In [ ]:
# ============================================================================
# MAIN EXECUTION CELL
# ============================================================================

logger.info("="*70)
logger.info("          ARC Grid Forensics Toolkit - Solver Execution")
logger.info("="*70)

# Configuration
DATA_ROOT = "/kaggle/input"  # Kaggle default input path
SUBMISSION_FILE = "submission.json"
DATASET_NAME = "arc-agi_test_challenges.json"  # Change as needed

start_time = time.time()

try:
    # 1. Initialize Solver
    solver = ArcSolver(data_root=DATA_ROOT, submission_path=SUBMISSION_FILE)
    
    # 2. Load Data
    if solver.load_data(dataset_name=DATASET_NAME):
        
        # 3. Run Solver Loop (Train and Predict)
        solver.run_solver(max_attempts_per_case=1, visualize_first=True)
        
        # 4. Generate Submission File
        submission_path = solver.generate_submission()
        elapsed_time = time.time() - start_time
        
        logger.info(f"⏱️  Execution finished in {elapsed_time:.2f} seconds.")
        logger.info(f"📄 Final Submission Path: {submission_path}")
        
    else:
        logger.error("❌ Skipping solver run due to data loading failure.")

except Exception as e:
    logger.critical(f"💥 A CRITICAL ERROR occurred in the main execution loop: {e}", exc_info=True)
    
logger.info("="*70)

---
## Cell 7: Testing and Debugging (Optional)
**Use this cell to test individual components or debug specific tasks**

In [ ]:
# ============================================================================
# TESTING AND DEBUGGING CELL
# ============================================================================

# Example: Test grid_forensics on a simple grid
test_grid = [
    [1, 1, 0, 0, 0],
    [1, 1, 0, 0, 0],
    [0, 0, 0, 0, 0],
    [0, 0, 0, 2, 0],
    [0, 0, 0, 0, 0]
]

print("\n🔍 Testing Grid Forensics...")
analysis = GridAnalysis(test_grid, background_color=0)

print(f"Objects found: {analysis.object_count}")
for obj in analysis.objects:
    print(f"  {obj}")
    print(f"  Shape Hash: {obj.shape_hash}")

display_grid(test_grid, "Test Grid")

# Example: Test a single task
# if solver.challenges:
#     task_id = list(solver.challenges.keys())[0]
#     task = solver.challenges[task_id]
#     print(f"\n🎯 Testing Task: {task_id}")
#     learned = solver.engine.train_task(task)
#     if learned:
#         test_input = task.test[0].input
#         prediction = solver.engine.solve_test_case(test_input)
#         display_grid(test_input, f"Input {task_id}")
#         display_grid(prediction, f"Prediction {task_id}")

---
## Cell 8: Add More Rules Here
**Template for adding new transformation rules**

In [ ]:
# ============================================================================
# CUSTOM RULES - Add your own transformation rules here
# ============================================================================

# Example: IdentityRule (passthrough)
class IdentityRule(BaseRule):
    """Simple passthrough rule for testing."""
    RULE_ID = "Identity_R00"
    
    def learn(self, task: ArcTask) -> bool:
        """Always returns True for testing."""
        self.is_learned = True
        return True
    
    def apply(self, input_grid: List[List[int]]) -> Optional[List[List[int]]]:
        """Returns the input unchanged."""
        return input_grid

# To use this rule, add it to solver.available_rules:
# solver.available_rules.append(IdentityRule)
# solver.engine = HypothesisEngine(rule_classes=solver.available_rules)

print("✅ Custom rules cell ready for your implementations")